## Setup

In [1]:
# Make sure to have the server side running in CoppeliaSim: 
# in a child script of a CoppeliaSim scene, add following command
# to be executed just once, at simulation start:
#
# simRemoteApi.start(19999)
#
# then start simulation, and run this program.
#
# IMPORTANT: for each successful call to simxStart, there
# should be a corresponding call to simxFinish at the end!

try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')

import time
import numpy as np

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim in synchronous

# if clientID!=-1:
#     print ('Connected to remote API server')

#     # enable the synchronous mode on the client:
#     sim.simxSynchronous(clientID,True)

#     # start the simulation:
#     sim.simxStartSimulation(clientID, sim.simx_opmode_blocking)
#     print('Synchronous mode enabled')
# else:
#     print ('Failed connecting to remote API server')  

Program started


## Robot Initialization

In [2]:
leftMotor1 = -1
leftMotor2 = -1
rightMotor1 = -1
rightMotor2 = -1

minSpeed = 50*np.pi/180
maxSpeed = 300*np.pi/180

startTime = 0

if clientID!=-1:
    print ('Connected to remote API server')

    # Now try to retrieve handles in a blocking fashion (i.e. a service call):
    
    # Initialize Motor
    err, leftMotor1 =sim.simxGetObjectHandle(clientID, '/Motor_FL', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_mot")
    err, leftMotor2 =sim.simxGetObjectHandle(clientID, '/Motor_BL', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_mot")
    err, rightMotor1 =sim.simxGetObjectHandle(clientID, '/Motor_FR', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_mot")
    err, rightMotor2 =sim.simxGetObjectHandle(clientID, '/Motor_BR', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_mot")
    
    # Initialize Sensors
    
    sensorHandle=[-1,-1,-1,-1,-1]
    err, sensorHandle[0] = sim.simxGetObjectHandle(clientID, '/Vis_0', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_0")    
    err, sensorHandle[1] = sim.simxGetObjectHandle(clientID, '/Vis_1', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_1")    
    err, sensorHandle[2] = sim.simxGetObjectHandle(clientID, '/Vis_2', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_2")    
    err, sensorHandle[3] = sim.simxGetObjectHandle(clientID, '/Vis_3', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_3")    
    err, sensorHandle[4] = sim.simxGetObjectHandle(clientID, '/Vis_4', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_4")
        
    err, res, _ = sim.simxReadVisionSensor(clientID, sensorHandle[0], sim.simx_opmode_streaming) 
    if(err != 0):
        print("IM_0")
    err, res, _ = sim.simxReadVisionSensor(clientID, sensorHandle[1], sim.simx_opmode_streaming)
    if(err != 0):
        print("IM_1") 
    err, res, _ = sim.simxReadVisionSensor(clientID, sensorHandle[2], sim.simx_opmode_streaming) 
    if(err != 0):
        print("IM_2") 
    err, res, _ = sim.simxReadVisionSensor(clientID, sensorHandle[3], sim.simx_opmode_streaming) 
    if(err != 0):
        print("IM_3") 
    err, res, _ = sim.simxReadVisionSensor(clientID, sensorHandle[4], sim.simx_opmode_streaming)
    if(err != 0):
        print("IM_4") 

    time.sleep(2)
    
    print("Done Initialization")

else:
    print ('Failed connecting to remote API server')

Connected to remote API server
IM_0
IM_1
IM_2
IM_3
IM_4
Done Initialization


In [3]:
sensorReading = np.array([0, 0, 0, 0, 0]).astype(float)

speed = (minSpeed + maxSpeed)/2


def readSensor():
    
    for i in range(5):
        err, state, value =sim.simxReadVisionSensor(clientID, sensorHandle[i], sim.simx_opmode_buffer) # Try to retrieve the streamed data
        sensorReading[i] = value[0][11]<0.5
    print(sensorReading)
    
 
    
def motorControl():
    
    rightV = speed
    leftV = speed
    
    if(sensorReading[0]):
        leftV=0.03*speed
    if(sensorReading[4]):
        rightV=0.03*speed

    sim.simxSetJointTargetVelocity(clientID, leftMotor1, leftV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, leftMotor2, leftV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, rightMotor1, rightV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, rightMotor2, rightV, sim.simx_opmode_oneshot)
    
        
if clientID!=-1:
    print ("Running LFR Routine")
     
    startTime = time.time()
    
    while time.time()-startTime < 100:
        
        readSensor()
        motorControl()
        
else:
    print ('Failed connecting to remote API server')


Running LFR Routine
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1

[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 1. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0

[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0

[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0

[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0

...

[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0

[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[1. 1. 0. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0

[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0.]
[0. 1. 1. 0. 0

[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0.]
[0. 0. 1. 1. 0

IndexError: list index out of range

In [ ]:
sim.simxFinish(clientID)

    
print ('Program ended')